In [1]:
# Import all packages needed in this notebook.
import numpy as np
import cv2 as cv
from PIL import Image
from matplotlib import pyplot as plt
import os
import glob
from scipy import signal
from scipy.ndimage import convolve
from skimage import restoration
import shutil
import albumentations as aug
from torchvision.models.resnet import resnet152
from torchvision.models.vgg import vgg19
from torchvision.models import VGG19_Weights, ResNet152_Weights
from torchvision.transforms import functional as F
import torch
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import wget

if not os.path.exists('resources'):
    raise TypeError("Please create the resources folder and include all required stuff")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\hseyyedehsan\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


TypeError: Please create the resources folder and include all required stuff

In [ ]:
def read_Img(input_img_path):
    if not isinstance(input_img_path, str):
        raise ValueError("Input must be a string")
    img_Orig = cv.imread(input_img_path)
    if img_Orig is None or img_Orig.ndim != 3:
        raise ValueError("Input image must have 3 channels")
    return img_Orig

In [ ]:
#Check images of the dataset###############here
########################################################################
def transform_image(input_image):
    input_image_rgb = cv.cvtColor(input_image, cv.COLOR_BGR2RGB)
    print('The size of image 1 ==>> ', input_image_rgb.shape)
    input_image_grayscale = cv.cvtColor(input_image_rgb, cv.COLOR_RGB2GRAY)
    return input_image_rgb, input_image_grayscale

In [ ]:
#Image Augmentation-Horizontal flip
def create_augmentation(image, i):
    transform = aug.HorizontalFlip(p=0.7)
    transformed = transform(image=image)  
    transformed_image = transformed["image"]  # Accessing correctly
    cv.imwrite(f"augmented_image_{i}.png", transformed_image)  
    #training dataset.A directory must be created in the main code script

In [ ]:
# Low pass filtering of images
def low_pass_filtering(image):
    kernel = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16.0
    conv_img1 = convolve(image, kernel, mode='constant')
    return conv_img1


In [ ]:
#Resizing function of the image. 
#Add a for loop to the main function of the script code toapply this to all images of the dataset
def resize_image(image, h, w):
    inter_polat = cv.INTER_CUBIC
    resized_image = cv.resize(image, (h, w), interpolation=inter_polat)
    return resized_image

In [ ]:
#Prepare data images for training
def arrange_dataset_for_training():
    imdir_train = "resources/dataset/images/"
    label_train = "resources/dataset/labels/"
    images_path = sorted(glob.glob(imdir_train + '*.jpg'))
    label_path = sorted(glob.glob(label_train + '*.txt'))
    X_train, X_second, y_train, y_second = train_test_split(images_path, label_path, test_size=0.20, random_state=123)
    X_valid, X_test, y_valid, y_test = train_test_split(X_second, y_second, test_size=0.25, random_state=123)
    return X_train, X_second, y_train, y_second, X_valid, X_test, y_valid, y_test

In [2]:
def video_loading(path_Video):
    os.makedirs("grayscale_images", exist_ok=True)
    os.makedirs("rgb_images", exist_ok=True)
    
    cap = cv.VideoCapture(path_Video)
    no_Frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    print(f'Total number of frames: {no_Frames}')
    print(f'The size of each frame: {frame_width} X {frame_height}')
    
    i = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print('No frames grabbed!')
            break
        image_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        cv.imwrite(f"rgb_images/rgb_frame{i}.jpg", image_rgb)
        image_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        cv.imwrite(f"grayscale_images/grayscale_frame{i}.jpg", image_gray)
        i += 1
    cap.release()
    cv.destroyAllWindows()


In [3]:
def tracking_visualization(path_frame, i): 
    # Visualize the results of the tracking model on the video frames
    os.makedirs("annotated_images", exist_ok=True)
    
    ############################################################################################  
    # Declare the model path that was trained
    model = YOLO("model_path.pt")   
    ############################################################################################  

    results = model.track(path_frame, persist=True)
    
    if results and len(results) > 0:  # Ensure results contain at least one frame
        annotated_frame = results[0].plot()
        cv.imwrite(f"annotated_images/object-tracked_frame{i}.jpg", annotated_frame)
    else:
        print(f"No objects tracked in frame {i}.")

